In [1]:
from glob import glob
from os.path import join, basename

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [11]:
common_path = "/Volumes/MMIS-Saraiv/Datasets/Newcastle/EC/sLORETA/LORETA_0,5Hz-resolution"
#common_path = "/Volumes/MMIS-Saraiv/Datasets/Sapienza/sLORETA/LORETA_0,5Hz-resolution"
out_common_path = "/Volumes/MMIS-Saraiv/Datasets/Newcastle/EC/features_source_ind-bands"
header = ['Frontal', 'Central', 'Parietal', 'Occipital', 'Temporal', 'Limbic']
freqs = np.arange(0.5, 45.5, 0.5)  # these are the lower bound frequencies of 0.5Hz bins

In [12]:
# Read IAF and TF values
iaf_tf_path = "/Volumes/MMIS-Saraiv/Datasets/Newcastle/EC/iaf_tf_my-boudaries.csv"
#iaf_tf_path = "/Volumes/MMIS-Saraiv/Datasets/Izmir/iaf_tf_my-boudaries.csv"
all_iaf_tf = pd.read_csv(iaf_tf_path, index_col=0)

In [13]:
all_subjects = glob(join(common_path, '*Roi.txt'))

In [14]:
def plot_individual_bands(bands: dict[str, tuple[float, float]], tf: float, iaf: float, subject_code: str):
    # Make a horizonal plot with the bands on the x-axis
    fig, ax = plt.subplots(1, 1, figsize=(20, 3))
    for i, (band, (inf, sup)) in enumerate(bands.items()):
        # coarse line
        plt.hlines(i, inf, sup, label=band, color='black', lw=4)
    # plot vertically dashed in red the TF and IAF
    plt.vlines(tf, -1, len(bands), color='red', linestyles='dashed', lw=2, label='TF')
    plt.vlines(iaf, -1, len(bands), color='blue', linestyles='dashed', lw=2, label='IAF')
    ax.set_yticks(range(len(bands)))
    ax.set_yticklabels(list(bands.keys()))
    ax.set_xlabel("Frequency (Hz)")
    # make all x-ticks from 'bands' and add TF and IAF
    xticks = list(bands.values())
    xticks.append((tf, tf))
    xticks.append((iaf, iaf))
    ax.set_xticks([x for x, _ in xticks])
    ax.set_title("Individual bands of subject " + subject_code)
    #plt.show()
    plt.savefig(join(out_common_path, f"{subject_code}_bands.png"), dpi=300, bbox_inches='tight')
    plt.close()

In [15]:
def average_band(_df, inf, sup, label, verbose=False):
    if verbose:
        print("Inf:", inf, "Sup:", sup)
        
    # Discretize input
    # If inf or sup are not .0 or .5, round to the nearest .0 or .5
    inf = round(inf*2)/2
    sup = round(sup*2)/2
    
    # Average by column
    res = _df.loc[inf: sup-0.5].mean(axis=0)
    
    # Print the index of all rows that were averaged
    bins = _df.loc[inf: sup-0.5].index.tolist()
    n_bins = len(bins)
    if verbose:
        print(label, "| Averaging", bins)
        
    return res, pd.Series({l: n_bins for l in header})

In [16]:
def flatten_df(_df):
    # E.g. Delta_Frontal, Delta_Central, Delta_Parietal, Delta_Occipital, Delta_Temporal, Delta_Limbic, ...
    _res = {}
    for band, band_df in _df.items():
        for region, value in band_df.items():
            _res[f"{band}_{region}"] = value
    _res = pd.DataFrame(_res, index=[0])
    return _res

In [17]:
res = pd.DataFrame(dtype=float)
res_number_bins = pd.DataFrame(dtype=int)
skipped_subjects = 0
low_tf_subjects = 0
for filepath in all_subjects:
    subject_code = basename(filepath).split('-')[0].split(' ')[1]
    #subject_code = basename(filepath).split('-')[0].split('PARTICIPANT')[1]
    print("Subject code:", subject_code)
    subject_out_filepath = join(out_common_path, f"{subject_code}.csv")
    
    # Read .txt file to Dataframe
    df = pd.read_csv(filepath, sep='  ', header=None, engine='python')
    df.columns = header
    df.index = freqs
    assert df.shape == (90, 6)
    
    # Get IAF and TF of this subject
    iaf, tf = all_iaf_tf.loc[int(subject_code)]
    print("IAF:", iaf, "TF:", tf)
    iaf, tf = round(iaf, 1), round(tf, 1)  # round to 1 decimal place
    
    # Extract/average values per individual band
    band_features = {}
    band_bins = {}
    
    if tf >= iaf:
        print("TF is greater than or equal to IAF. Skipping...")
        skipped_subjects += 1
        continue
     
    if tf < 4.5:
        print("TF is less than 4Hz, hence no Delta band")#, Skipping...")
        low_tf_subjects += 1
        #continue
    
    # Delta
    delta_inf = tf - 4 if tf - 4 >= 2 else 2
    delta_sup = tf-2 if tf-2 > delta_inf else 3
    band_features['Delta'], band_bins['Delta'] = average_band(df, delta_inf, delta_sup, 'Delta', verbose=True)
    # Alpha 1
    alpha1_sup = tf + ((iaf - tf) / 2)
    alpha1_inf = tf if alpha1_sup - tf >= 0.5 else alpha1_sup - 0.5
    if alpha1_inf < 6:
        alpha1_inf = 6 
        alpha1_sup = 6.5
    band_features['Alpha1'], band_bins['Alpha1'] = average_band(df, alpha1_inf, alpha1_sup, 'Alpha1', verbose=True)
    # Theta
    theta_inf = tf-2 if tf-2 >= delta_sup else delta_sup
    theta_sup = tf if tf - theta_inf >= 2 else alpha1_inf
    if theta_sup > alpha1_inf:
        theta_sup = alpha1_inf
    band_features['Theta'], band_bins['Theta'] = average_band(df, theta_inf, theta_sup, 'Theta', verbose=True)
    # Alpha 2
    alpha2_inf = alpha1_sup
    alpha2_sup = iaf if iaf - alpha2_inf >= 0.5 else alpha2_inf + 1.0
    band_features['Alpha2'], band_bins['Alpha2'] = average_band(df, alpha2_inf, alpha2_sup, 'Alpha2', verbose=True)
    # Alpha 3
    alpha3_inf = iaf if iaf >= alpha2_sup else alpha2_sup
    alpha3_sup = iaf + 2 if iaf + 2 <= 13 else 13
    band_features['Alpha3'], band_bins['Alpha3'] = average_band(df, alpha3_inf, alpha3_sup, 'Alpha3')
    # Beta 1
    beta1_inf = 14
    beta1_sup = 20
    band_features['Beta1'], band_bins['Beta1'] = average_band(df, beta1_inf, beta1_sup, 'Beta1')
    # Beta 2
    beta2_inf = 20
    beta2_sup = 30
    band_features['Beta2'], band_bins['Beta2'] = average_band(df, beta2_inf, beta2_sup, 'Beta2')
    # Gamma
    gamma_inf = 30
    gamma_sup = 40
    band_features['Gamma'], band_bins['Gamma'] = average_band(df, gamma_inf, gamma_sup, 'Gamma')
    
    # Plot individual bands
    plot_individual_bands({'Delta': (delta_inf, delta_sup), 'Theta': (theta_inf, theta_sup), 'Alpha1': (alpha1_inf, alpha1_sup), 
                           'Alpha2': (alpha2_inf, alpha2_sup), 'Alpha3': (alpha3_inf, alpha3_sup), 'Beta1': (beta1_inf, beta1_sup)},
                          tf, iaf, subject_code)
    
    # Flatten band_features
    subject_res = flatten_df(band_features)
    subject_res.index = [subject_code]
    
    subject_res_bins = flatten_df(band_bins)
    subject_res_bins.index = [subject_code] 
    
    # Add to res using concat
    res = pd.concat([res, subject_res], axis=0)
    res_number_bins = pd.concat([res_number_bins, subject_res_bins], axis=0)

In [18]:
print("Skipped subjects:", skipped_subjects)
print("Low TF subjects:", low_tf_subjects)

In [19]:
res

In [20]:
res_number_bins

In [21]:
# Save res
res.to_csv(join(out_common_path, "all_features.csv"))